# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import pickle
import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, balanced_accuracy_score, precision_score, recall_score

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yesil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yesil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yesil\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\yesil\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('DisasterMessageTable', engine)
X = df['message'].values
Y = df[df.columns[4:]].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    stop_words = stopwords.words("english")
    
    text = re.sub(r"[a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = [lemmatizer.lemmatize(tok).strip() for tok in tokens if tok not in stop_words]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline_forest = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [6]:
pipeline_forest.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline_forest.predict(X_test)

In [8]:
target_names = df.columns[4:]

for i in range(len(target_names)):
    print(target_names[i], ':')
    print(classification_report(y_test[:,i], y_pred[:,i]))

related :
              precision    recall  f1-score   support

           0       0.48      0.07      0.12      1261
           1       0.76      0.97      0.85      3929
           2       0.17      0.02      0.04        46

    accuracy                           0.75      5236
   macro avg       0.47      0.36      0.34      5236
weighted avg       0.69      0.75      0.67      5236

request :
              precision    recall  f1-score   support

           0       0.84      0.98      0.91      4359
           1       0.46      0.09      0.15       877

    accuracy                           0.83      5236
   macro avg       0.65      0.53      0.53      5236
weighted avg       0.78      0.83      0.78      5236

offer :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5220
           1       0.00      0.00      0.00        16

    accuracy                           1.00      5236
   macro avg       0.50      0.50      0.50   

C:\Users\yesil\anaconda3\envs\Udacity_env\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4987
           1       0.12      0.01      0.02       249

    accuracy                           0.95      5236
   macro avg       0.53      0.50      0.50      5236
weighted avg       0.91      0.95      0.93      5236

electricity :
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5131
           1       0.00      0.00      0.00       105

    accuracy                           0.98      5236
   macro avg       0.49      0.50      0.49      5236
weighted avg       0.96      0.98      0.97      5236

tools :
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5208
           1       0.00      0.00      0.00        28

    accuracy                           0.99      5236
   macro avg       0.50      0.50      0.50      5236
weighted avg       0.99      0.99      0.99      523

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline_forest.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [10]:
parameters_forest = {
            'vect__ngram_range': ((1, 1), (1, 2)),
            'vect__max_df': (0.5, 0.75, 1.0),
            'clf__estimator__n_estimators': [20, 50],
            }

cv_forest = GridSearchCV(pipeline_forest, param_grid=parameters_forest, n_jobs=1, verbose=2)
cv_forest.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1), total=  21.2s
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.1s remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1), total=  20.6s
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1), total=  21.9s
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1), total=  21.4s
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 1), total=  21.0s
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 2), total=  38.5s
[CV] clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=20, vect__max_df=0.5, vect__ngram_range=(1, 2), total=  37.9s
[CV] clf

[CV]  clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 2), total= 1.3min
[CV] clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 2), total= 1.2min
[CV] clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_estimators=50, vect__max_df=0.75, vect__ngram_range=(1, 2), total= 1.3min
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, vect__ngram_range=(1, 1), total=  57.6s
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, vect__ngram_range=(1, 1), total=  56.9s
[CV] clf__estimator__n_estimators=50, vect__max_df=1.0, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_estimators=50, vect__max_df=1.0, vect__ngram_range=(1, 1), total=  57.4s
[CV

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 46.2min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [11]:
cv_forest.best_params_

{'clf__estimator__n_estimators': 50,
 'vect__max_df': 0.5,
 'vect__ngram_range': (1, 1)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
y_pred_forest = cv_forest.predict(X_test)

In [18]:
def model_performance(y_true, y_predicted):
    accuracy_list = []
    precision_list = []
    recall_list = []
    for i in range(len(target_names)):
        accuracy_list.append(balanced_accuracy_score(y_true[:,i], y_pred[:,i]))
        precision_list.append(precision_score(y_true[:,i], y_pred[:,i], average='weighted'))
        recall_list.append(recall_score(y_true[:,i], y_pred[:,i], average='weighted'))

    return np.mean(accuracy_list), np.mean(precision_list), np.mean(recall_list)

In [19]:
acc_forest, precision_forest, recall_forest = model_performance(y_test, y_pred_forest)
print('Model performance:\naccuracy: {}\nprecisicion: {}\nrecall: {}'
      .format(acc_forest, precision_forest, recall_forest))

Model performance:
accuracy: 0.9258604957134369
precisicion: 0.8963166766430276
recall: 0.9258604957134369


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
pipeline_kNN = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

In [21]:
pipeline_kNN.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__algorithm', 'clf__estimator__leaf_size', 'clf__estimator__metric', 'clf__estimator__metric_params', 'clf__estimator__n_jobs', 'clf__estimator__n_neighbors', 'clf__estimator__p', 'clf__estimator__weights', 'clf__estimator', 'clf__n_jobs'])

In [22]:
parameters_kNN = {
            'vect__ngram_range': ((1, 1), (1, 2)),
            'vect__max_df': (0.5, 0.75, 1.0),
            'clf__estimator__n_neighbors': [2, 4, 6],
            }

cv_kNN = GridSearchCV(pipeline_kNN, param_grid=parameters_kNN, n_jobs=1, verbose=2)
cv_kNN.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1), total=  59.6s
[CV] clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   59.5s remaining:    0.0s


[CV]  clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1), total= 1.0min
[CV] clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1), total= 1.1min
[CV] clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1), total= 1.2min
[CV] clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 1), total= 1.4min
[CV] clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 2), total= 1.5min
[CV] clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_neighbors=2, vect__max_df=0.5, vect__ngram_range=(1, 2), total= 1.5min
[CV] clf__estimator__n_neighbo

[CV]  clf__estimator__n_neighbors=4, vect__max_df=0.75, vect__ngram_range=(1, 2), total= 1.4min
[CV] clf__estimator__n_neighbors=4, vect__max_df=0.75, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__n_neighbors=4, vect__max_df=0.75, vect__ngram_range=(1, 2), total= 1.6min
[CV] clf__estimator__n_neighbors=4, vect__max_df=1.0, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_neighbors=4, vect__max_df=1.0, vect__ngram_range=(1, 1), total= 2.2min
[CV] clf__estimator__n_neighbors=4, vect__max_df=1.0, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_neighbors=4, vect__max_df=1.0, vect__ngram_range=(1, 1), total= 2.3min
[CV] clf__estimator__n_neighbors=4, vect__max_df=1.0, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_neighbors=4, vect__max_df=1.0, vect__ngram_range=(1, 1), total= 2.3min
[CV] clf__estimator__n_neighbors=4, vect__max_df=1.0, vect__ngram_range=(1, 1) 
[CV]  clf__estimator__n_neighbors=4, vect__max_df=1.0, vect__ngram_range=(1, 1), total= 2.3min
[CV] clf__estimator__n_neig

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 158.2min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [23]:
y_pred_kNN = cv_kNN.predict(X_test)

In [26]:
acc_kNN, precision_kNN, recall_kNN = model_performance(y_test, y_pred_kNN)
print('Model performance:\naccuracy: {}\nprecisicion: {}\nrecall: {}'
      .format(acc_kNN, precision_kNN, recall_kNN))

Model performance:
accuracy: 0.9258604957134369
precisicion: 0.8963166766430276
recall: 0.9258604957134369


C:\Users\yesil\anaconda3\envs\Udacity_env\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 9. Export your model as a pickle file

In [27]:
with open('random_forest.pkl', 'wb') as f:
    pickle.dump(cv_forest, f)

with open('kNN.pkl', 'wb') as f:
    pickle.dump(cv_kNN, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.